In [15]:
import requests
import os
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
ratio_key = os.getenv("RATIO_API_KEY")
company_name=input('Company name: ')
Company_profile=requests.get(f'https://financialmodelingprep.com/api/v3/search?query={company_name}&apikey={ratio_key}')
symbol=Company_profile.json()[0]['symbol']
print(symbol)

APC.F


In [17]:
output = ""
print(f'Company Name: {company_name}')
output+='this is the data of '+ company_name
# Liquidity ratios
print(' Liquidity Ratios:')
# output += ' ,Liquidity Ratios:'
Balance_Sheet=requests.get(f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{symbol}?period=annual&apikey={ratio_key}')
# leverage_financial_ratio Current ratio = Current assets / Current liabilities
if not Balance_Sheet.json():
    print('No Data')

else:
    current_assets=Balance_Sheet.json()[0]['totalCurrentAssets']
    current_liabilities=Balance_Sheet.json()[0]['totalCurrentLiabilities']
    current_ratio=current_assets/current_liabilities
    date=Balance_Sheet.json()[0]['date']
    print(date)
    output += f", {date}"
    print('Current Ratio: '+ str(current_ratio))
    output += f", Current Ratio: {current_ratio}"

    if 1.2 <= current_ratio <= 2:
        print("A current ratio of", current_ratio, "is considered good. The business has enough current assets to cover its short-term liabilities.")
        output += f", A current ratio of {current_ratio} is considered good. The business has enough current assets to cover its short-term liabilities."
    elif current_ratio < 1:
        print("A current ratio of", current_ratio, "is below 1. This means that the company may not have enough liquid assets to cover its short-term liabilities.")
        output += f", A current ratio of {current_ratio} is below 1. This means that the company may not have enough liquid assets to cover its short-term liabilities."
    else:
        print("The current ratio of", current_ratio, "is within an acceptable range, but it's not considered optimal.")
        output += f", A current ratio of {current_ratio} is within an acceptable range, but it's not considered optimal."
# Acid Test Ratio current assets - inventories/current liabilties "totalCurrentLiabilities"
inventory=Balance_Sheet.json()[0]['inventory']    
atr = (current_assets - inventory) / current_liabilities
print('Acid Test Ratio: ' + str(atr))
output += f", Acid Test Ratio: {atr}"
if atr > 1:
    print("An acid test ratio of", atr, "means that the company does have enough liquid assets to cover its short-term debts")
    output +=" ,An acid test ratio of", atr, "means that the company does have enough liquid assets to cover its short-term debts"
elif atr < 1:
    print("An acid test ratio of", atr, "means that the company does not have enough liquid assets to cover its short-term debts")
    
else:
    print("The acid test ratio", atr, "means that the company has just enough to cover its short-term debts")
    output +=" ,The acid test ratio", atr, "means that the company has just enough to cover its short-term debts"

# operatingCashFlow (opcf) = Operating cash flow / Current liabilities    
Cash_Flow=requests.get(f'https://financialmodelingprep.com/api/v3/cash-flow-statement/{symbol}?period=annual&apikey={ratio_key}')
opc=Cash_Flow.json()[0]["operatingCashFlow"]
opcf=opc/current_liabilities
print("Operating Cash Flow: "+ str(opcf))
output += f", Operating Cash Flow: {opcf}"
if opcf < 1:
    print ('An operating Cash Flow of less than one indicates that a company s core business operations are not generating enough cash to cover its current operating expenses.')
    output +=' ,An operating Cash Flow of less than one indicates that a company s core business operations are not generating enough cash to cover its current operating expenses.'
elif opcf > 1:
    print("OCF greater than 1 is a positive indicator for a company's financial health. It signifies that the company's core business operations are generating enough cash to cover its current operating expenses")
    output +=" ,OCF greater than 1 is a positive indicator for a company's financial health. It signifies that the company's core business operations are generating enough cash to cover its current operating expenses"
else:
    print(" company's core business operations are generating just enough cash to cover its current operating expenses.")
    output +=" ,company's core business operations are generating just enough cash to cover its current operating expenses."

# Profitability Ratios
print('Profitability Ratios:')
output +=' ,Profitability Ratios:'
# Gross margin ratio = Gross profit / Net sales
Income_statement=requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{symbol}?period=annual&apikey={ratio_key}')
GPR=Income_statement.json()[0]["grossProfitRatio"]
GPR=GPR*100
print("Gross Profit Ratio: "+ str(GPR)+' %')
output +=" Gross Profit Ratio: "+ str(GPR)+' %'

Ratos=requests.get(f'https://financialmodelingprep.com/api/v3/ratios/{symbol}?apikey={ratio_key}')
data=Ratos.json()[0]
output+='this is the data of '+ company_name
output += f",  {data}"
print(output)                                                                                 

Company Name: apple
 Liquidity Ratios:
2023-09-30
Current Ratio: 0.9880116717592975
A current ratio of 0.9880116717592975 is below 1. This means that the company may not have enough liquid assets to cover its short-term liabilities.
Acid Test Ratio: 0.9444421504665951
An acid test ratio of 0.9444421504665951 means that the company does not have enough liquid assets to cover its short-term debts
Operating Cash Flow: 0.7607495802020535
An operating Cash Flow of less than one indicates that a company s core business operations are not generating enough cash to cover its current operating expenses.
Profitability Ratios:
Gross Profit Ratio: 44.13112958 %
this is the data of apple, 2023-09-30, Current Ratio: 0.9880116717592975, A current ratio of 0.9880116717592975 is below 1. This means that the company may not have enough liquid assets to cover its short-term liabilities., Acid Test Ratio: 0.9444421504665951, Operating Cash Flow: 0.7607495802020535 ,An operating Cash Flow of less than one 

In [18]:
print(company_name)
print(date)
def generate_conclusion(output):
    api_key = os.getenv("OPENAI_API_KEY")
    
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    
    data = {
        "model": "gpt-3.5-turbo",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f" use all the ratios and catograize them with that give a review of the financial health of the company : {output}"}
        ],
        "max_tokens": 1, 
        "temperature": 0.5 
    }
    
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=data)
    
    if response.status_code == 200:
        try:
            conclusion = response.json()['choices'][0]['message']['content']
            return conclusion
        except KeyError as e:
            raise KeyError(f"Expected key not found in response: {e}")
    else:
        try:
            error_message = response.json().get('error', {}).get('message', 'Unknown error')
        except ValueError:
            error_message = "Failed to parse error message from response."
        raise Exception(f"Error: {response.status_code}, {error_message}")

try:
    conclusion = generate_conclusion(output)
    print(conclusion)
except Exception as e:
    print(f"An error occurred: {e}")


apple
2023-09-30
Based
